In [1]:
import pandas as pd
from datetime import datetime

rawpath = '../raw'

In [3]:
df = pd.read_excel(f'{rawpath}/Plakate_1933-45_ImportCBS.xlsx', names=['themengruppe', 'laendercode','entstehungszeit','konvolut4105','konvolut6710','signatur','schon_erschlossen'])
df

,themengruppe,laendercode,entstehungszeit,konvolut4105,konvolut6710,signatur,schon_erschlossen
0,SU/Ukraine,NaN,1941-1942,1180323777,!1246329107!,WKII 15.6.3,keine
1,NaN,/1XA-SUHH,1941-1942,!1180323777!,!1246329107!,WKII 15.6.3.1,NaN
2,NaN,NaN,1941-1942,!1180323777!,!1246329107!,WKII 15.6.3.2,NaN
3,NaN,NaN,1941-1942,!1180323777!,!1246329107!,WKII 15.6.3.3,NaN
4,NaN,NaN,1941-1942,!1180323777!,!1246329107!,WKII 15.6.3.4,NaN
...,...,...,...,...,...,...,...
1229,Belgien,NaN,1939-1943,!117658510X!,!124632749X!,WKII 15.3.1.655,NaN
1230,Belgien,NaN,1939-1943,!117658510X!,!124632749X!,WKII 15.3.1.656,NaN
1231,Belgien,NaN,1939-1943,!117658510X!,!124632749X!,WKII 15.3.1.657,NaN
1232,Belgien,NaN,1939-1943,!117658510X!,!124632749X!,WKII 15.3.1.658,NaN


In [7]:
def pica_schreiben(row):
    pica = f"""\t\n0500 Aaf
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy\n"""
    
    if len(str(row['entstehungszeit'])) > 0 and len(str(row['entstehungszeit'])) < 5:
        # nur ein datum
        pica += f"1100 {row['entstehungszeit']}\n1110 *{row['entstehungszeit']}$4ezth\n"
    elif len(str(row['entstehungszeit'])) > 4 and len(str(row['entstehungszeit'])) < 10:
        pica += f"1100 {row['entstehungszeit'][:4]}$b{row['entstehungszeit'][5:]}\n1110 *{row['entstehungszeit']}$a{row['entstehungszeit'][:4]}$b{row['entstehungszeit'][5:]}$4ezth\n"
        # wohl zwei daten
    else:
        pica += "\n\n1100 FEHLER\n\n"
    
    pica += f"""1130 TB-papier
1130 !040445224!
1131 !04046198X!
1700 {row['laendercode']}
2105 04,P01
4000 {row['signatur']}\n"""
    # 4030 todo
    # if pd.isna(row['verlagsort']):
        # pica += "4030 [Ort nicht ermittelbar] : [Verlag nicht ermittelbar]\n"
    # else:
        # pica += f"4030 [{row['verlagsort']}] : [Verlag nicht ermittelbar]\n"
    
    pica += f"""4060 1 ungezähltes Blatt
4105 !1122570686!
4105 {row['konvolut4105']}
4700 |BSM|
7001 {now.strftime('%d-%m-%y')} : x
6710 {row['konvolut6710'].strip()}
6800 [Provenienz]
6800 !004709608! *1961-1993
6800 Stempel
7100 GS PS WKII @ m
7109 !!DBSM/S!! ; GS PS {row['signatur']}\n"""
    return pica


In [8]:
now = datetime.now()
with open(f"../dat/wkII-{now.strftime('%y-%m-%d-%H-%M-%S')}-test.dat","w") as f:
    for index, row in df.sample(5).iterrows():
        f.write(pica_schreiben(row))